<div style="padding:20px;color:white;margin:0;font-size:175%;text-align:center;display:fill;border-radius:5px;background-color:#016CC9;overflow:hidden;font-weight:500">CREDIT CARD FRAUD</div>

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>1 | Preprocessing Steps</div></b>

### <b><span style='color:#DEB078'>1.1 |</span><span style='color:#016CC9'> Import Libraries</span></b>  

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from sklearn import model_selection,preprocessing,metrics,linear_model
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

### <b><span style='color:#DEB078'>1.2 |</span><span style='color:#016CC9'> Configure Settings</span></b>    

In [ ]:
%matplotlib inline
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)          # Set now of columns visible to 100
temp=dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), height=500, width=1000))
pd.set_option('display.max_rows',100)          # Set now of rows visible to 100

### <b><span style='color:#DEB078'>1.3 |</span><span style='color:#016CC9'> Import Files</span></b>    

In [ ]:
df_org   = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
df_train = pd.read_csv("/kaggle/input/playground-series-s3e4/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s3e4/test.csv")

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>2 | Dataframe Investigation</div></b>

### <b><span style='color:#DEB078'>2.1 |</span><span style='color:#016CC9'> Structure Validation & Merging</span></b> 

In [ ]:
df_org.shape

In [ ]:
df_train.shape

In [ ]:
#Original
print('Column Present only in original ', set(df_org.columns).difference(set(df_train.columns)))
print('Column Present only in train    ', set(df_train.columns).difference(set(df_org.columns)))

In [ ]:
df_train['id'].nunique() # its a row identifier 

In [ ]:
df_train.drop(columns='id',inplace=True)

In [ ]:
# Validating
print('Column Present only in original ', set(df_org.columns).difference(set(df_train.columns)))
print('Column Present only in train    ', set(df_train.columns).difference(set(df_org.columns)))

In [ ]:
df_org.loc[:,'src']='Training Data'
df_org.head(2)

In [ ]:
df_train.loc[:,'src']='Original Data'
df_train.head(2)

In [ ]:
# Merge both into 1 
df=pd.concat([df_org,df_train]).reset_index(drop=True)

In [ ]:
pd.value_counts(df.dtypes)

In [ ]:
# No Null values present
df.info()

In [ ]:
df.head(2)

### <b><span style='color:#DEB078'>2.2 |</span><span style='color:#016CC9'> Null Value Check</span></b> 
3 Columns are having null value present

In [ ]:
plt.subplots(1,2,figsize=(13,3))
plt.subplot(1,2,1)
plt.imshow(df.isna(), aspect="auto", interpolation="nearest", cmap="gray") # we can test any condition in df using this graph
plt.xlabel("Column Number")
plt.ylabel("Sample Number")
plt.title("DATASET VIEW - Black (Not Null) & While (Null)")
plt.subplot(1,2,2)
df.isna().mean().sort_values().plot( kind="bar", title="Null Value Ratio per Feature", ylabel="Ratio of missing values per feature")
plt.show()

### <b><span style='color:#DEB078'>2.3 |</span><span style='color:#016CC9'> Unique Vallue Check</span></b> 

#### <b><span style='color:#DEB078'>2.3.1 |</span><span style='color:#016CC9'> Remove Duplicate Rows</span></b> 

In [ ]:
df.shape

In [ ]:
df=df.drop_duplicates(subset=df.columns.tolist()[:-1], keep='first').reset_index(drop=True)

In [ ]:
df.shape

#### <b><span style='color:#DEB078'>2.3.2 |</span><span style='color:#016CC9'> Observe Unique in Columns</span></b> 

In [ ]:
pd.DataFrame(df.nunique()).T

In [ ]:
# Class should be categorical Variable
df[['Class']]=df[['Class']].astype('str')

In [ ]:
# Combined Plot

plt.subplots(1,2,figsize=(14,4),squeeze=False)
plt.subplot(1,2,1)
df.select_dtypes(include="number").nunique().sort_values().plot.bar(title="Unique values per Numerical feature");
plt.subplot(1,2,2)
df.select_dtypes(exclude="number").nunique().sort_values().plot.bar(title="Unique values per Categorical feature");
plt.tight_layout()
plt.show()

In [ ]:
# For all Numerical columnn for all rows 
df.plot(lw=0, marker=".", subplots=True, layout=(-1, 3),
          figsize=(12,10), markersize=5,color='#016CC9')
plt.tight_layout()

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>3 | Univariate Analysis</div></b>  

### <b><span style='color:#DEB078'>3.2 |</span><span style='color:#016CC9'> Numerical Columns</span></b>

In [ ]:
# Plots the histogram for each numerical feature in a separate subplot
figsize_val=(13,9)
layout_val=(-1, 4)
df.hist(bins=30, figsize=figsize_val, layout=layout_val, edgecolor="black",color='#DEB078')
plt.tight_layout();

In [ ]:
df.skew()

In [ ]:
vars=df.columns.tolist()[:-1]
y=4

x=math.ceil(len(df.columns)/y)

plt.subplots(x,y,figsize=(14,12))

for i in range(1,len(vars)) :
    plt.subplot(x,y,i)
    sns.boxplot(data=df,y=vars[i-1],x='src',palette=['#016CC9','#DEB078'])
    #plt.xlabel(vars[i-1])
    #plt.title(vars[i-1])
plt.tight_layout()    
plt.show()

In [ ]:
kk=(df.select_dtypes(include='number').columns).to_list()

df2=df[kk]


upper =df2.quantile(0.75)+1.5 *(df2.quantile(0.75)-df2.quantile(0.25))
lower = df2.quantile(0.25)-1.5 *(df2.quantile(0.75)-df2.quantile(0.25))

for i in kk:
    print('\nColumn -> ',i,'')   
    x= df2[ (df2[i]<round(lower[i],2)) |  (df2[i]>round(upper[i],2))][i].to_list()
    print('No of Outliers presesnt -> ',len(x))
    print('\n','*'*10)

### <b><span style='color:#DEB078'>3.2 |</span><span style='color:#016CC9'> Target Variable</span></b> 

In [ ]:
df[['Class']].value_counts(normalize=True).round(decimals=3)*100

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>4 | Multivariate Analysis</div></b>    

### <b><span style='color:#DEB078'>4.1 |</span><span style='color:#016CC9'> Creating Relation Matrix and Graph</span></b>

In [ ]:
temp_corr=df.corr()

In [ ]:
# Plot
plt.figure(figsize=(18,12))
sns.heatmap(df.corr().round(2), xticklabels=df.corr().columns, yticklabels=df.corr().columns, cmap='RdYlGn', center=0, annot=True)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

### <b><span style='color:#DEB078'>4.2 |</span><span style='color:#016CC9'> Relation With Target Variable (Class)</span></b>

In [ ]:
y=5
x=math.ceil((len(df.columns)-1)/y)-1

plt.subplots(x,y,figsize=(12,12))

for i in range(1,len(vars)) :
    plt.subplot(x,y,i)
    sns.boxplot(data=df,y=vars[i-1],x='Class',palette=['#016CC9','#DEB078'])
plt.tight_layout()    
plt.show()

In [ ]:
pd.plotting.radviz(df[vars], "Class",color=['#016CC9','#DEB078'])

### <b><span style='color:#DEB078'>4.3 |</span><span style='color:#016CC9'> Other Relationships (Leaving Class)</span></b> 

In [ ]:
temp_corr=df.corr()
res = [(a, b) for idx, a in enumerate(temp_corr.index) for b in temp_corr.index[idx + 1:]]

strong=[]
moderate=[]
weak=[]
no_relationship=[]
for x,y in res:
    p=temp_corr[x][y]
    if p >= 0.75 or p <= -0.75:
        strong.append((x,y))
    elif p >= 0.50 or p <= -0.50:
        moderate.append((x,y))
    elif p >= 0.25 or p <= -0.25:
        weak.append((x,y))
    else :
        no_relationship.append((x,y))
    
print('\nStrong -> ',strong)
print('\nModerate -> ',moderate)
print('\nWeak -> ',weak)

In [ ]:
data=df.sample(frac=0.01,random_state=3) 
tt=['Amount','Time','V2','V3','V5','V7','V20','Class']

sns.pairplot(data=data[tt] ,
             hue='Class',
             palette=['#016CC9','#DEB078'],
             corner=True
)
                  
plt.show()

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>5 | Feature Engineering</div></b>      

### <b><span style='color:#DEB078'>5.1 |</span><span style='color:#016CC9'> Drop Src Column</span></b>  

In [ ]:
df=df.drop(columns='src')
df_train=df_train.drop(columns='src')

### <b><span style='color:#DEB078'>5.2 |</span><span style='color:#016CC9'> Variable Splits</span></b>   

In [ ]:
target_Variable     = 'Class'
input_features      = [col for col in df.columns if col not in ['kfold',target_Variable]]
# numerical_columns   = ['age', 'fnlwgt','capital.gain', 'capital.loss','hours.per.week']
# categorical_columns = [col for col in input_features if not col in numerical_columns]

### <b><span style='color:#DEB078'>5.3 |</span><span style='color:#016CC9'> Create Folds</span></b>   

In [ ]:
def create_stratified_folds(data_df,no_of_splits,target_column):
    
    data_df['kfold']=-1
    
    #data_df=data_df.sample(frac=1,random_state=3) # random state to generate same set (seed)(for reproducibility.)
    
    kf=model_selection.StratifiedKFold(n_splits=no_of_splits,shuffle=True, random_state=10)
    
    for fold,(train_indexes,test_indexes) in enumerate(kf.split(data_df,data_df[target_column].values)):
        data_df.loc[test_indexes,'kfold']=np.int16(fold)
   
    return data_df

### <b><span style='color:#DEB078'>5.4 |</span><span style='color:#016CC9'> Handle Numerical Variables</span></b>

In [ ]:
def handle_numerical_variables(train_data,test_data,numerical_columns):
    scaler = preprocessing.StandardScaler()
    scaler.fit(train_data[[numerical_columns]])
    train_data[[numerical_columns]]=scaler.transform(train_data[[numerical_columns]]) 
    test_data[[numerical_columns]]=scaler.transform(test_data[[numerical_columns]])
    
    return train_data,test_data,scaler

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>6 | Model With Train Data</div></b>       

### <b><span style='color:#DEB078'>6.1 |</span><span style='color:#016CC9'> Logistic Regression With Train Data only </span></b> 

In [ ]:
lr_2_prediction=0

df1=create_stratified_folds(df_train,5,'Class')

average_auc=[]

for i in range(5):
    train_df = df1[df1['kfold']!=i]
    test_df  = df1[df1['kfold']==i]

    train_df,test_df,scaler = handle_numerical_variables(train_df,test_df,'Amount')

    x_train_df = train_df.drop(columns=['Class','kfold'])
    y_train_df = train_df.Class

    x_test_df = test_df.drop(columns=['Class','kfold'])
    y_test_df = test_df.Class

    # initialize Logistic Regression model
    model = linear_model.LogisticRegression()
    model.fit(x_train_df, y_train_df)

    valid_preds = model.predict_proba(x_test_df)[:, 1]
    auc = metrics.roc_auc_score(y_test_df, valid_preds)
    
    print(f'FOLD -> {i} | AUC -> {auc}')
    average_auc.append(auc)
    print('*'*15)
    
    test1=test.drop(columns=['id'])
    test1['Amount']=scaler.transform(test1[['Amount']])
    test_preds = model.predict_proba(test1)[:, 1]
    lr_2_prediction =lr_2_prediction+ test_preds/5
    
print('Average AUC value -> ',np.array(average_auc).mean())

### <b><span style='color:#DEB078'>6.2 |</span><span style='color:#016CC9'> XGBClassifier With Train Data only </span></b> 

In [ ]:
xgbparameters = {"objective":"binary:logistic","eval_metric": "auc","n_estimators": 1000,"random_state": 10,
    'tree_method': 'gpu_hist','predictor': 'gpu_predictor',
}


In [ ]:

xg_2_prediction=0
df1=create_stratified_folds(df_train,5,'Class')
df1.Class=df1.Class.astype(np.int16)
average_auc=[]

for i in range(5):
    train_df = df1[df1['kfold']!=i]
    test_df  = df1[df1['kfold']==i]

    x_train_df = train_df.drop(columns=['Class','kfold'])
    y_train_df = train_df.Class

    x_test_df = test_df.drop(columns=['Class','kfold'])
    y_test_df = test_df.Class

    # initialize XGBClassifier model
    model = XGBClassifier(**xgbparameters)
    model.fit(x_train_df, y_train_df,verbose=0)
    
    valid_preds = model.predict_proba(x_test_df)[:, 1]
    auc = metrics.roc_auc_score(y_test_df,  valid_preds)
    average_auc.append(auc)
    
    print(f'FOLD -> {i} | AUC -> {auc}')
    print('*'*15)
    
    test_preds = model.predict_proba(test.drop(columns=['id']))[:, 1]
    xg_2_prediction =xg_2_prediction+ test_preds/5
    
print('Average AUC value -> ',np.array(average_auc).mean())

### <b><span style='color:#DEB078'>6.3 |</span><span style='color:#016CC9'> GaussianNB With Train Data only </span></b> 

In [ ]:
df1=create_stratified_folds(df_train,5,'Class')
gb_2_prediction=0
average_auc=[]

for i in range(5):
    train_df = df1[df1['kfold']!=i]
    test_df  = df1[df1['kfold']==i]


    x_train_df = train_df.drop(columns=['Class','kfold'])
    y_train_df = train_df.Class

    x_test_df = test_df.drop(columns=['Class','kfold'])
    y_test_df = test_df.Class

    # initialize GaussianNB model
    model = GaussianNB()
    model.fit(x_train_df, y_train_df)

    valid_preds = model.predict_proba(x_test_df)[:, 1]
    auc = metrics.roc_auc_score(y_test_df, valid_preds)
    
    print(f'FOLD -> {i} | AUC -> {auc}')
    average_auc.append(auc)
    print('*'*15)
    
    test1=test.drop(columns=['id'])
    test_preds = model.predict_proba(test1)[:, 1]
    gb_2_prediction =gb_2_prediction+ test_preds/5
    
print('Average AUC value -> ',np.array(average_auc).mean())

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>7 | Model With Original and Train Data</div></b>       

### <b><span style='color:#DEB078'>7.1 |</span><span style='color:#016CC9'> Logistic Regression Combined (Org and Train)</span></b> 

In [ ]:
df1=create_stratified_folds(df,5,'Class')
lr_1_prediction=0
average_auc=[]

for i in range(5):
    train_df = df1[df1['kfold']!=i]
    test_df  = df1[df1['kfold']==i]

    train_df,test_df,scaler = handle_numerical_variables(train_df,test_df,'Amount')

    x_train_df = train_df.drop(columns=['Class','kfold'])
    y_train_df = train_df.Class

    x_test_df = test_df.drop(columns=['Class','kfold'])
    y_test_df = test_df.Class

    # initialize Logistic Regression model
    model = linear_model.LogisticRegression()
    model.fit(x_train_df, y_train_df)

    valid_preds = model.predict_proba(x_test_df)[:, 1]
    auc = metrics.roc_auc_score(y_test_df, valid_preds)
    
    print(f'FOLD -> {i} | AUC -> {auc}')
    average_auc.append(auc)
    print('*'*15)
    
    test1=test.drop(columns=['id'])
    test1['Amount']=scaler.transform(test1[['Amount']])
    test_preds = model.predict_proba(test1)[:, 1]
    lr_1_prediction =lr_1_prediction+ test_preds/5
    
print('Average AUC value -> ',np.array(average_auc).mean())

### <b><span style='color:#DEB078'>7.2 |</span><span style='color:#016CC9'> XGBClassifier Combined (Org and Train) </span></b> 

In [ ]:

xg_1_prediction=0
df1=create_stratified_folds(df,5,'Class')
df1.Class=df1.Class.astype(np.int16)
average_auc=[]

for i in range(5):
    train_df = df1[df1['kfold']!=i]
    test_df  = df1[df1['kfold']==i]

    x_train_df = train_df.drop(columns=['Class','kfold'])
    y_train_df = train_df.Class

    x_test_df = test_df.drop(columns=['Class','kfold'])
    y_test_df = test_df.Class

    # initialize XGBClassifier model
    model = XGBClassifier(**xgbparameters)
    model.fit(x_train_df, y_train_df,verbose=0)
    
    valid_preds = model.predict_proba(x_test_df)[:, 1]
    auc = metrics.roc_auc_score(y_test_df,  valid_preds)
    average_auc.append(auc)
    
    print(f'FOLD -> {i} | AUC -> {auc}')
    print('*'*15)
    
    test_preds = model.predict_proba(test.drop(columns=['id']))[:, 1]
    xg_1_prediction =xg_1_prediction+ test_preds/5
    
print('Average AUC value -> ',np.array(average_auc).mean())

### <b><span style='color:#DEB078'>7.3 |</span><span style='color:#016CC9'> GaussianNB Combined (Org and Train) </span></b> 

In [ ]:
df1=create_stratified_folds(df,5,'Class')
gb_1_prediction=0
average_auc=[]

for i in range(5):
    train_df = df1[df1['kfold']!=i]
    test_df  = df1[df1['kfold']==i]

    #train_df,test_df = handle_numerical_variables(train_df,test_df,'Amount')

    x_train_df = train_df.drop(columns=['Class','kfold'])
    y_train_df = train_df.Class

    x_test_df = test_df.drop(columns=['Class','kfold'])
    y_test_df = test_df.Class

    # initialize GaussianNB model
    model = GaussianNB()
    model.fit(x_train_df, y_train_df)

    valid_preds = model.predict_proba(x_test_df)[:, 1]
    auc = metrics.roc_auc_score(y_test_df, valid_preds)
    
    print(f'FOLD -> {i} | AUC -> {auc}')
    average_auc.append(auc)
    print('*'*15)
    
    test1=test.drop(columns=['id'])
    test_preds = model.predict_proba(test1)[:, 1]
    gb_2_prediction =gb_1_prediction+ test_preds/5
    
print('Average AUC value -> ',np.array(average_auc).mean())

In [ ]:
submission = pd.read_csv("/kaggle/input/playground-series-s3e4/sample_submission.csv")
submission['Class'] = xg_1_prediction
submission.to_csv("submit.csv",index=False)

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%'>8 | Other Notebooks</div></b>       

Please feel free to go through my other notebooks and provide your feedback
* https://www.kaggle.com/code/abhi011097/detailed-eda-basic-model-spaceship-titanic
* https://www.kaggle.com/code/abhi011097/learn-practice-pyspark-pandas-sql
* https://www.kaggle.com/abhi011097/code